In [ ]:
#data path acquisition
from glob import glob 

#data preprocessing and computation
import numpy as np
import pandas as pd
import cv2

#progress bars for tracking processes
from tqdm import tqdm 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#FILE PATHS
TRAIN_CSV = '/content/gdrive/MyDrive/Severstal/train.csv'
TRAIN_IMGS = '/content/gdrive/MyDrive/Severstal/train_images/'
TEST_CSV = '/content/gdrive/MyDrive/Severstal/sample_submission.csv'

##Training data

In [ ]:
train_df_origin = pd.read_csv(TRAIN_CSV)
train_df_origin.shape

(7095, 3)

In [ ]:
train_full = pd.Series(sorted(glob(TRAIN_IMGS + '*.jpg')))
train_full = train_full.str.split('/', expand=True)
print(train_full.shape)
train_full.head()

(12568, 7)


,0,1,2,3,4,5,6
0,,content,gdrive,MyDrive,Severstal,train_images,0002cc93b.jpg
1,,content,gdrive,MyDrive,Severstal,train_images,00031f466.jpg
2,,content,gdrive,MyDrive,Severstal,train_images,000418bfc.jpg
3,,content,gdrive,MyDrive,Severstal,train_images,000789191.jpg
4,,content,gdrive,MyDrive,Severstal,train_images,0007a71bf.jpg


In [ ]:
train_df = pd.DataFrame(columns=['ImageId_ClassId','EncodedPixels'])
for i in tqdm(range(len(train_full))):
    for j in range(4):
        tmp_se = pd.Series( [train_full[6][i]+'_{}'.format(j+1),None], index=train_df.columns )
        train_df = train_df.append( tmp_se, ignore_index=True )
train_df.head()

100%|██████████| 12568/12568 [02:03<00:00, 101.74it/s]


,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,None
1,0002cc93b.jpg_2,None
2,0002cc93b.jpg_3,None
3,0002cc93b.jpg_4,None
4,00031f466.jpg_1,None


In [ ]:
for i in tqdm(range(len(train_df_origin))):
    imageid_classid = train_df_origin['ImageId'][i]+'_{}'.format(train_df_origin['ClassId'][i])
    idx=train_df.query('ImageId_ClassId == "{}"'.format(imageid_classid)).index[0]
    train_df['EncodedPixels'][idx] = train_df_origin['EncodedPixels'][i]
train_df.head()

100%|██████████| 7095/7095 [00:36<00:00, 192.83it/s]


,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,None
2,0002cc93b.jpg_3,None
3,0002cc93b.jpg_4,None
4,00031f466.jpg_1,None


In [ ]:
train_df['ImageId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
train_df['ClassId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[1])
train_df['hasMask'] = ~ train_df['EncodedPixels'].isna()

# print(train_df.shape)
train_df.head()

,ImageId_ClassId,EncodedPixels,ImageId,ClassId,hasMask
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...,0002cc93b.jpg,1,True
1,0002cc93b.jpg_2,None,0002cc93b.jpg,2,False
2,0002cc93b.jpg_3,None,0002cc93b.jpg,3,False
3,0002cc93b.jpg_4,None,0002cc93b.jpg,4,False
4,00031f466.jpg_1,None,00031f466.jpg,1,False


In [ ]:
maskCount_df = train_df.groupby('ImageId').agg(np.sum).reset_index()
maskCount_df.sort_values('hasMask', ascending=False, inplace=True)
maskCount_df['hasMask'] = maskCount_df['hasMask'].astype(float)
maskCount_df.head()

,ImageId,hasMask
10803,db4867ee8.jpg,3.0
11776,ef24da2ba.jpg,3.0
6284,7f30b9c64.jpg,2.0
9421,bf0c81db6.jpg,2.0
9615,c314f43f3.jpg,2.0


##Submission Data

In [ ]:
sub_df_origin = pd.read_csv('/content/gdrive/MyDrive/Severstal/sample_submission.csv')
sub_df_origin.head()

,ImageId,EncodedPixels,ClassId
0,0000f269f.jpg,1 409600,0
1,000ccc2ac.jpg,1 409600,0
2,002451917.jpg,1 409600,0
3,003c5da97.jpg,1 409600,0
4,0042e163f.jpg,1 409600,0


In [ ]:
sub_df = pd.DataFrame(columns=['ImageId_ClassId','EncodedPixels'])
for i in tqdm(range(len(sub_df_origin))):
    for j in range(4):
        tmp_se = pd.Series( [sub_df_origin['ImageId'][i]+'_{}'.format(j+1),None], index=sub_df.columns )
        sub_df = sub_df.append( tmp_se, ignore_index=True )
sub_df.head(9)

100%|██████████| 5506/5506 [00:41<00:00, 133.00it/s]


,ImageId_ClassId,EncodedPixels
0,0000f269f.jpg_1,None
1,0000f269f.jpg_2,None
2,0000f269f.jpg_3,None
3,0000f269f.jpg_4,None
4,000ccc2ac.jpg_1,None
5,000ccc2ac.jpg_2,None
6,000ccc2ac.jpg_3,None
7,000ccc2ac.jpg_4,None
8,002451917.jpg_1,None


In [ ]:
sub_df['ImageId'] = sub_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
test_imgs = pd.DataFrame(sub_df['ImageId'].unique(), columns=['ImageId'])

In [ ]:
train_df.to_csv('train_df.csv', index =False)
maskCount_df.to_csv('maskCount_df.csv', index=False)
sub_df.to_csv('sub_df.csv', index=False)
test_imgs.to_csv('test_imgs.csv', index=False)